# kaggle study 18일차(Statoil)

코드출처 : https://www.kaggle.com/devm2024/transfer-learning-with-vgg-16-cnn-aug-lb-0-1712

TL;DR입니다.
GPU에서 실행 CPU와의 호환성 문제가 있습니다.

1. 딥 러닝의 하이퍼 파라미터는 여러 가지가 있으며, 이를 조정하려면 몇 주 또는 몇 달이 걸립니다. 일반적으로 연구자들은 이 튜닝을 수행하고 다른 아키텍처보다 성능이 뛰어난 멋진 아키텍처 세트를 발견하면 논문을 발표합니다.

2. 사전 교육을 받은 모델이기 때문에 매우 빠르게 수렴되고 사용자도 GPU를 사용해야 합니다. 일부 라이브러리 문제로 인해 CPU에서 작동하지 않습니다.

3. 우리의 목적을 위해, 우리는 연구원들이 이용할 수 있는 그런 건축물을 사용할 수 있습니다.

4. 이미 기능을 추출하는 방법을 알고 있는 미리 훈련된 네트워크를 사용하면 하이퍼 파라미터를 조정할 필요가 없습니다. 이미 일부 데이터 세트(이미지넷이라고 함)에 대해 교육을 받았기 때문에 미리 훈련된 가중치는 가중치의 초기화를 잘 제공하며, 이로 인해 우리의 Convnet은 매우 빠르게 수렴되며 그렇지 않으면 이러한 심층 아키텍처에 며칠이 걸릴 수 있습니다. 그것이 Transfer Learning의 아이디어입니다. 그 예로는 VGG16, InceptionNet, google, Resnet 등이 있습니다.

5. 이 커널에서는 작은 크기의 이미지에서 매우 우수한 성능을 발휘하는 사전 훈련된 VGG-16 네트워크를 사용할 것입니다.

VGG 아키텍처가 작은 크기의 이미지에서도 잘 작동하는 것으로 입증되었습니다(CIFAR-10). 이 데이터 세트에서도 잘 작동할 것으로 예상했습니다.

1. 이 코드에는 데이터 확대 단계도 포함되어 있으므로 성능이 상당히 개선됩니다.

2. GPU가 필요합니다.

관심이 있으시다면 여기 연구 논문의 링크가 있습니다. https://arxiv.org/pdf/1409.1556.pdf

또한 keras 라이브러리(https://keras.io/applications/#vg16)도 있습니다.

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [3]:
train = pd.read_json('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/statoil/train.json')
target_train=train['is_iceberg']
test = pd.read_json('C:/Users/이동훈/Desktop/github/kaggle/kagglestudy/Data/statoil/test.json')

Keras는 사전 훈련된 VGG의 구현을 제공합니다. 그것은 라이브러리에 있으므로 우리는 직접 네트워크를 구축할 필요가 없습니다. 여기서는 VGG의 마지막 레이어를 제거하고 이진 예측을 위한 시그모이드 레이어를 배치합니다.

다음 코드는 작동하지 않습니다. 카글 노트북에서는 모델의 가중치를 다운로드할 수 없지만, 자신의 노트북에 코드를 복사하여 붙여넣을 수 있습니다.

In [8]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

#교육 데이터를 생성
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=(X_band_1+X_band_2)/2
#X_band_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in train["inc_angle"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis]
                          , X_band_2[:, :, :, np.newaxis]
                         , X_band_3[:, :, :, np.newaxis]], axis=-1)



X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=(X_band_test_1+X_band_test_2)/2
#X_band_test_3=np.array([np.full((75, 75), angel).astype(np.float32) for angel in test["inc_angle"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , X_band_test_3[:, :, :, np.newaxis]], axis=-1)

from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input	

from keras.preprocessing.image import ImageDataGenerator
batch_size=64

# 이미지 변환을 정의
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# 두 generator를 병합하도록 함.
# 우리는 동일한 랜덤 시드를 가진 정확히 동일한 생성기를 그들과 각도 배열에 사용
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #어시스트 어레이는 동일합니다. 이는 마음의 평화를 위한 것이었지만 훈련 속도는 느려짐
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

def get_callbacks(filepath, patience=2):
   es = EarlyStopping('val_loss', patience=10, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    

    x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(512, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(512, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model([base_model.input, input_2],predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#데이터 확대와 함께 K-폴드 교차 검증을 사용.
def myAngleCV(X_train, X_angle, X_test):
    K=3
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #파일 경로를 지정하고 콜백을 받음.
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #제일 좋은 모델 구하기
        galaxyModel.load_weights(filepath=file_path)
        #train 스코어 구하기
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #test 스코어 구하기
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #validation 스코어 구하기
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #test 스코어 구하기
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])
    
        #train 스코어 구하기
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [9]:
preds=myAngleCV(X_train, X_angle, X_test)


===================FOLD= 0


C:\Users\이동훈\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
24/24 [==============================] - 200s 8s/step - loss: 1.0535 - accuracy: 0.5648 - val_loss: 0.4676 - val_accuracy: 0.7738
Epoch 2/100
24/24 [==============================] - 101s 4s/step - loss: 0.5410 - accuracy: 0.7494 - val_loss: 0.3094 - val_accuracy: 0.8579
Epoch 3/100
24/24 [==============================] - 98s 4s/step - loss: 0.3659 - accuracy: 0.8290 - val_loss: 0.2948 - val_accuracy: 0.8673
Epoch 4/100
24/24 [==============================] - 97s 4s/step - loss: 0.3031 - accuracy: 0.8709 - val_loss: 0.2736 - val_accuracy: 0.8766
Epoch 5/100
24/24 [==============================] - 98s 4s/step - loss: 0.2916 - accuracy: 0.8666 - val_loss: 0.2807 - val_accuracy: 0.8804
Epoch 6/100
24/24 [==============================] - 99s 4s/step - loss: 0.2903 - accuracy: 0.8686 - val_loss: 0.2377 - val_accuracy: 0.8916
Epoch 7/100
24/24 [==============================] - 98s 4s/step - loss: 0.2559 - accuracy: 0.8746 - val_loss: 0.2368 - val_accuracy: 0.8972
Epoch 8/100

24/24 [==============================] - 112s 5s/step - loss: 0.2104 - accuracy: 0.9073 - val_loss: 0.2788 - val_accuracy: 0.8858
Epoch 7/100
24/24 [==============================] - 111s 5s/step - loss: 0.2197 - accuracy: 0.8959 - val_loss: 0.2530 - val_accuracy: 0.8951
Epoch 8/100
24/24 [==============================] - 111s 5s/step - loss: 0.2047 - accuracy: 0.9181 - val_loss: 0.2874 - val_accuracy: 0.8783
Epoch 9/100
24/24 [==============================] - 111s 5s/step - loss: 0.2031 - accuracy: 0.9214 - val_loss: 0.2783 - val_accuracy: 0.8933
Epoch 10/100
24/24 [==============================] - 119s 5s/step - loss: 0.2000 - accuracy: 0.9208 - val_loss: 0.2632 - val_accuracy: 0.9064
Epoch 11/100
24/24 [==============================] - 119s 5s/step - loss: 0.1830 - accuracy: 0.9300 - val_loss: 0.2978 - val_accuracy: 0.8783
Epoch 12/100
24/24 [==============================] - 125s 5s/step - loss: 0.1761 - accuracy: 0.9301 - val_loss: 0.3029 - val_accuracy: 0.8858
Epoch 13/100
24

In [10]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('sub.csv', index=False)